<a href="https://colab.research.google.com/github/shahinul22/machine-learning-pretrained-model-/blob/main/EfficientNetB0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()  # Select your dataset.zip file
!unzip dataset.zip -d dataset/


Saving dataset.zip to dataset.zip
Archive:  dataset.zip
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0365.JPG  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0366.jpg  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0367.JPG  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0370.jpg  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0372.JPG  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0373.JPG  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0374.JPG  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0375.JPG  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0376.JPG  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0377.JPG  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0378.JPG  
  inflating: dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0379.JPG  
  inflating: dataset

In [ ]:
!pip install tensorflow keras opencv-python matplotlib scikit-learn


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_data = train_datagen.flow_from_directory(
    "/content/dataset/rice_leaf_diseases",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

val_data = train_datagen.flow_from_directory(
    "/content/dataset/rice_leaf_diseases",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 96 images belonging to 3 classes.
Found 24 images belonging to 3 classes.


In [ ]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # Freeze pretrained layers

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(train_data.num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 46s 7s/step - accuracy: 0.3828 - loss: 1.1321 - val_accuracy: 0.3333 - val_loss: 1.1056
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 887ms/step - accuracy: 0.2565 - loss: 1.1649 - val_accuracy: 0.3333 - val_loss: 1.1355
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 849ms/step - accuracy: 0.2852 - loss: 1.1832 - val_accuracy: 0.3333 - val_loss: 1.1136
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.3451 - loss: 1.1304 - val_accuracy: 0.3333 - val_loss: 1.1182
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 843ms/step - accuracy: 0.2826 - loss: 1.1449 - val_accuracy: 0.3333 - val_loss: 1.1004
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 851ms/step - accuracy: 0.3073 - loss: 1.1354 - val_accuracy: 0.3333 - val_loss: 1.1077
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 895ms/step - accuracy: 0.4141 - loss: 1.0957 - val_accuracy: 0.3333 - val_loss: 1.1047
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 879ms/step - accuracy: 0.3177 - loss: 1.1071 - val_accuracy: 0.3333 - val_loss: 1.09

In [ ]:
model.save("/content/drive/MyDrive/rice_disease_efficientnetb0.h5")


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import os

# Load the saved model
model = load_model("/content/drive/MyDrive/rice_disease_efficientnetb0.h5")


# Assuming you want to test with an image from the unzipped dataset
# Let's pick an example image path from the unzip output
img_path = "/content/dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0702.jpg" # Example path, change if needed

# Check if the file exists before attempting to load
if not os.path.exists(img_path):
    print(f"Error: Image file not found at {img_path}")
else:
    img = image.load_img(img_path, target_size=(224,224))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    class_idx = np.argmax(pred)

    # Get the class labels from the training data generator
    # You need access to train_data.class_indices which was created in cell 2sB0HQA-FE6G
    # Assuming train_data is available in the environment
    if 'train_data' in globals():
        disease = list(train_data.class_indices.keys())[class_idx]
        print("Predicted Disease:", disease)
    else:
        print("Error: train_data object not found. Cannot retrieve class labels.")
        print("Prediction probabilities:", pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
Predicted Disease: Brown spot


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import os

# Load the saved model
model = load_model("/content/drive/MyDrive/rice_disease_efficientnetb0.h5")


# Assuming you want to test with an image from the unzipped dataset
# Let's pick an example image path from the unzip output
img_path = "/content/dataset/rice_leaf_diseases/Bacterial leaf blight/DSC_0702.jpg" # Example path, change if needed

# Check if the file exists before attempting to load
if not os.path.exists(img_path):
    print(f"Error: Image file not found at {img_path}")
else:
    img = image.load_img(img_path, target_size=(224,224))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    class_idx = np.argmax(pred)

    # Get the class labels from the training data generator
    # You need access to train_data.class_indices which was created in cell 2sB0HQA-FE6G
    # Assuming train_data is available in the environment
    if 'train_data' in globals():
        disease = list(train_data.class_indices.keys())[class_idx]
        print("Predicted Disease:", disease)
    else:
        print("Error: train_data object not found. Cannot retrieve class labels.")
        print("Prediction probabilities:", pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Predicted Disease: Brown spot


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("/content/drive/MyDrive/rice_disease_efficientnetb0.h5")

# Specify the directory containing the images you want to test
test_image_dir = "/content/dataset/rice_leaf_diseases/Leaf smut" # Change this to the directory you want to test

# Get the class labels from the training data generator (assuming it's still in the environment)
if 'train_data' in globals():
    class_labels = list(train_data.class_indices.keys())
else:
    print("Error: train_data object not found. Cannot retrieve class labels.")
    class_labels = None # Or load labels from a saved file

# Iterate through the images in the directory
if os.path.exists(test_image_dir):
    for filename in os.listdir(test_image_dir):
        if filename.endswith((".jpg", ".jpeg", ".png")): # Add other image extensions if needed
            img_path = os.path.join(test_image_dir, filename)

            try:
                img = image.load_img(img_path, target_size=(224, 224))
                img_array = image.img_to_array(img) / 255.0
                img_array = np.expand_dims(img_array, axis=0)

                pred = model.predict(img_array)
                class_idx = np.argmax(pred)

                if class_labels:
                    predicted_disease = class_labels[class_idx]
                    print(f"Image: {filename}, Predicted Disease: {predicted_disease}")
                else:
                    print(f"Image: {filename}, Prediction probabilities: {pred}")

            except Exception as e:
                print(f"Error processing image {filename}: {e}")
else:
    print(f"Error: Test image directory not found at {test_image_dir}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
Image: DSC_0328.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Image: DSC_0516.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image: DSC_0509.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Image: DSC_0514.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Image: DSC_0503.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Image: DSC_0512.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image: DSC_0336.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image: DSC_0506.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Image: DSC_0507.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image: DSC_0339.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Image: DSC_0510.jpg, Predicted Disease: Brown spot
1/1 ━━━━━━━━━━━━━━━━━━━

In [ ]:
# Check GPU
!nvidia-smi

# Install libraries (Colab has most already, but this is safe)
!pip install -q tensorflow matplotlib scikit-learn opencv-python


Fri Aug 22 09:55:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   76C    P0             30W /   70W |    2168MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os, shutil, random, math
random.seed(42)

# Path where class folders live (update if needed)
DATA_ROOT = "/content/dataset"

# Detect if already split
if os.path.exists(os.path.join(DATA_ROOT, "train")) and os.path.exists(os.path.join(DATA_ROOT, "val")):
    print("Found existing train/val folders. Skipping split.")
    TRAIN_DIR = os.path.join(DATA_ROOT, "train")
    VAL_DIR   = os.path.join(DATA_ROOT, "val")
else:
    print("No split found. Creating train/val split (80/20).")
    ALL_DIR = DATA_ROOT  # class folders are directly inside DATA_ROOT
    SPLIT_DIR = "/content/data_split"
    TRAIN_DIR = os.path.join(SPLIT_DIR, "train")
    VAL_DIR   = os.path.join(SPLIT_DIR, "val")
    os.makedirs(TRAIN_DIR, exist_ok=True)
    os.makedirs(VAL_DIR, exist_ok=True)

    # classes = subfolders
    classes = [d for d in os.listdir(ALL_DIR) if os.path.isdir(os.path.join(ALL_DIR, d))]
    print("Classes:", classes)

    for cls in classes:
        src = os.path.join(ALL_DIR, cls)
        files = [f for f in os.listdir(src) if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))]
        random.shuffle(files)
        n_total = len(files)
        n_val = max(1, int(0.2 * n_total))  # 20% validation
        val_files = files[:n_val]
        train_files = files[n_val:]

        # make class dirs
        os.makedirs(os.path.join(TRAIN_DIR, cls), exist_ok=True)
        os.makedirs(os.path.join(VAL_DIR, cls), exist_ok=True)

        for f in train_files:
            shutil.copy2(os.path.join(src, f), os.path.join(TRAIN_DIR, cls, f))
        for f in val_files:
            shutil.copy2(os.path.join(src, f), os.path.join(VAL_DIR, cls, f))

    print("Split complete.")
    !find /content/data_split -maxdepth 2 -type d -print


No split found. Creating train/val split (80/20).
Classes: ['rice_leaf_diseases']
Split complete.
/content/data_split
/content/data_split/train
/content/data_split/train/rice_leaf_diseases
/content/data_split/val
/content/data_split/val/rice_leaf_diseases


In [ ]:
import tensorflow as tf
import json
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.utils.class_weight import compute_class_weight

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

# Augmentation & preprocessing (use preprocess_input for EfficientNet)
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=25,
    width_shift_range=0.08,
    height_shift_range=0.08,
    zoom_range=0.15,
    horizontal_flip=True
)
val_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_data = train_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=SEED
)
val_data = val_gen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

# Compute class weights to fix imbalance
y_train = train_data.classes  # array of class indices for each sample in train generator
classes = np.unique(y_train)
class_weights_arr = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weights = {i: w for i, w in enumerate(class_weights_arr)}
print("Class weights:", class_weights)

# Save label mapping (so you can load it later at inference time)
label_map = {cls_name: int(idx) for cls_name, idx in train_data.class_indices.items()}
with open("/content/drive/MyDrive/class_indices.json", "w") as f:
    json.dump(label_map, f, indent=2)
print("Saved label map to Drive.")



Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
!ls -R /content/dataset | head -50


/content/dataset:
rice_leaf_diseases

/content/dataset/rice_leaf_diseases:
Bacterial leaf blight
Brown spot
Leaf smut

/content/dataset/rice_leaf_diseases/Bacterial leaf blight:
DSC_0365.JPG
DSC_0366.jpg
DSC_0367.JPG
DSC_0370.jpg
DSC_0372.JPG
DSC_0373.JPG
DSC_0374.JPG
DSC_0375.JPG
DSC_0376.JPG
DSC_0377.JPG
DSC_0378.JPG
DSC_0379.JPG
DSC_0380.JPG
DSC_0381.JPG
DSC_0382.JPG
DSC_0383.JPG
DSC_0384.JPG
DSC_0385.jpg
DSC_0386.JPG
DSC_0388.JPG
DSC_0389.JPG
DSC_0390.JPG
DSC_0392.JPG
DSC_0393.JPG
DSC_0395.JPG
DSC_0396.JPG
DSC_0397.JPG
DSC_0398.JPG
DSC_0399.JPG
DSC_0400.JPG
DSC_0401.JPG
DSC_0402.JPG
DSC_0403.JPG
DSC_0404.JPG
DSC_0405.JPG
DSC_0406.JPG
DSC_0700.jpg
DSC_0701.jpg
DSC_0702.jpg
DSC_0703.JPG



In [ ]:
# =========================================
# STEP 0: Setup
# =========================================
!nvidia-smi   # check GPU
!pip install -q tensorflow matplotlib scikit-learn opencv-python

from google.colab import drive
drive.mount('/content/drive')

# =========================================
# STEP 1: Data pipeline (using validation_split)
# =========================================
import tensorflow as tf
import numpy as np, json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.utils.class_weight import compute_class_weight

IMG_SIZE = (224,224)
BATCH_SIZE = 32
DATASET_DIR = "/content/dataset/rice_leaf_diseases"

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=42
)

val_data = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

# Compute class weights (to fix imbalance)
y_train = train_data.classes
classes = np.unique(y_train)
class_weights_arr = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weights = {i: w for i, w in enumerate(class_weights_arr)}
print("Class weights:", class_weights)

# Save label mapping
label_map = {cls: idx for cls, idx in train_data.class_indices.items()}
with open("/content/drive/MyDrive/class_indices.json", "w") as f:
    json.dump(label_map, f, indent=2)
print("Saved label map:", label_map)

# =========================================
# STEP 2: Build EfficientNet-B0 Model
# =========================================
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

base = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224,224,3))
base.trainable = False   # freeze for stage 1

inputs = layers.Input(shape=(224,224,3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(train_data.num_classes, activation="softmax")(x)
model = models.Model(inputs, outputs)

model.compile(optimizer=optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

# =========================================
# STEP 3: Train (Stage 1)
# =========================================
ckpt_stage1 = "/content/drive/MyDrive/rice_efficientnetb0_stage1.keras"
cbs = [
    EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True),
    ModelCheckpoint(ckpt_stage1, monitor="val_accuracy", save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
]

history1 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    class_weight=class_weights,
    callbacks=cbs
)

# Save stage 1 model
model.save("/content/drive/MyDrive/rice_disease_efficientnetb0_stage1.keras")

# =========================================
# STEP 4: Fine-tune (unfreeze top layers)
# =========================================
for layer in base.layers[-40:]:   # unfreeze last 40 layers
    layer.trainable = True

model.compile(optimizer=optimizers.Adam(5e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

ckpt_final = "/content/drive/MyDrive/rice_efficientnetb0_final.keras"
cbs_ft = [
    EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True),
    ModelCheckpoint(ckpt_final, monitor="val_accuracy", save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
]

history2 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    class_weight=class_weights,
    callbacks=cbs_ft
)

# Save final model
model.save("/content/drive/MyDrive/rice_disease_efficientnetb0_final.keras")

# =========================================
# STEP 5: Evaluate
# =========================================
val_loss, val_acc = model.evaluate(val_data, verbose=0)
print(f"Final Validation Accuracy: {val_acc*100:.2f}%")


Fri Aug 22 10:02:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0             30W /   70W |    2168MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Model: "functional_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │         3,843 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,053,414 (15.46 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.3976 - loss: 1.1849
Epoch 1: val_accuracy improved from -inf to 0.79167, saving model to /content/drive/MyDrive/rice_efficientnetb0_stage1.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 29s 4s/step - accuracy: 0.4049 - loss: 1.1740 - val_accuracy: 0.7917 - val_loss: 0.7704 - learning_rate: 0.0010
Epoch 2/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 364ms/step - accuracy: 0.4983 - loss: 1.0330
Epoch 2: val_accuracy improved from 0.79167 to 0.91667, saving model to /content/drive/MyDrive/rice_efficientnetb0_stage1.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 855ms/step - accuracy: 0.5091 - loss: 1.0215 - val_accuracy: 0.9167 - val_loss: 0.7041 - learning_rate: 0.0010
Epoch 3/15
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step - accuracy: 0.6753 - loss: 0.8687
Epoch 3: val_accuracy did not improve from 0.91667
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 540ms/step - accuracy: 0.6810 - loss: 0.8569 - val_accuracy: 0.9167 - val_loss: 0.6372 - learning_rate: 0.0010
Epoch 4/15
3/3 ━━━━━━━━━━━━━━

In [ ]:
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load final model + labels
MODEL_PATH = "/content/drive/MyDrive/rice_disease_efficientnetb0_final.keras"
LABELS_JSON = "/content/drive/MyDrive/class_indices.json"

model = load_model(MODEL_PATH)
with open(LABELS_JSON) as f:
    label_map = json.load(f)
index_to_class = {v:k for k,v in label_map.items()}
print("Labels:", index_to_class)

# Test folder
test_folder = "/content/dataset/rice_leaf_diseases/Brown spot"

for fname in os.listdir(test_folder):
    if fname.lower().endswith((".jpg",".jpeg",".png")):
        path = os.path.join(test_folder, fname)
        img = image.load_img(path, target_size=(224,224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        probs = model.predict(x, verbose=0)[0]
        idx = int(np.argmax(probs))
        pred_class = index_to_class[idx]
        print(f"{fname:25s} → {pred_class:20s} ({probs[idx]:.3f}) | probs={np.round(probs,3)}")


Labels: {0: 'Bacterial leaf blight', 1: 'Brown spot', 2: 'Leaf smut'}
DSC_0104.jpg              → Brown spot           (0.443) | probs=[0.246 0.443 0.311]
DSC_0323.JPG              → Brown spot           (0.606) | probs=[0.26  0.606 0.134]
DSC_0391.jpg              → Brown spot           (0.540) | probs=[0.342 0.54  0.118]
DSC_0303.JPG              → Brown spot           (0.634) | probs=[0.218 0.634 0.148]
DSC_0302.JPG              → Brown spot           (0.406) | probs=[0.34  0.406 0.255]
DSC_0324.JPG              → Leaf smut            (0.408) | probs=[0.388 0.204 0.408]
DSC_0112.jpg              → Brown spot           (0.583) | probs=[0.29  0.583 0.127]
DSC_0119.jpg              → Brown spot           (0.656) | probs=[0.146 0.656 0.198]
DSC_0121.jpg              → Brown spot           (0.613) | probs=[0.189 0.613 0.198]
DSC_0111.jpg              → Brown spot           (0.639) | probs=[0.102 0.639 0.259]
DSC_0307.JPG              → Brown spot           (0.434) | probs=[0.403 0.434 0.